In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
words = open('names.txt', 'r').read().splitlines()

In [6]:
print(words[:8])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [52]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {s:i for i,s in stoi.items()}

In [78]:
# build dataset
block_size = 3
X, Y = [], []
for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix]
X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [79]:
X.shape, Y.shape, X.dtype, Y.dtype

(torch.Size([32, 3]), torch.Size([32]), torch.int64, torch.int64)

In [106]:
# C is a matrix of embeddings of dim = 2
# Each word (char in alphabet -> assoc. w/ dim2 vector)
C = torch.randn((27,2))
C[:2]

tensor([[ 0.2574,  0.7931],
        [ 1.3124, -0.0372]])

In [107]:
X[:2]

tensor([[0, 0, 0],
        [0, 0, 5]])

In [110]:
emb = C[X]
emb[:2]

tensor([[[ 0.2574,  0.7931],
         [ 0.2574,  0.7931],
         [ 0.2574,  0.7931]],

        [[ 0.2574,  0.7931],
         [ 0.2574,  0.7931],
         [-0.0430, -0.8601]]])

In [146]:
emb.view(-1,6)[:2] # yay lines up with what we want

tensor([[ 0.2574,  0.7931,  0.2574,  0.7931,  0.2574,  0.7931],
        [ 0.2574,  0.7931,  0.2574,  0.7931, -0.0430, -0.8601]])

In [154]:
W = torch.rand((6,100))
b = torch.rand(100)
h = emb.view(-1,6) @ W + b
h.shape

torch.Size([32, 100])

In [155]:
W2 = torch.rand((100,27))
b2 = torch.rand(27)
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [ ]:
counts = logits.